In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
import sklearn.preprocessing as skp
from sklearn.model_selection import train_test_split
import tensorflow.keras as tfk
import tensorflow as tf
from tensorflow.keras import layers
from sklearn.metrics import accuracy_score

2024-04-21 01:34:11.435090: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-04-21 01:34:11.439882: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-21 01:34:11.635737: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-21 01:34:12.327732: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-21 01:34:14.639645: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF

In [2]:
from sklearn.pipeline import Pipeline, make_pipeline, FunctionTransformer
from sklearn.impute import SimpleImputer

In [3]:
dataset = pd.read_csv("./diabetes.csv")

In [4]:
target = dataset["Outcome"]
features = dataset.drop(columns=["Outcome"],axis=1)

In [5]:
scaler = skp.StandardScaler()

In [6]:
model = tfk.Sequential([
    layers.InputLayer(shape=(6,)),
    layers.Dense(4,activation="relu"),
    layers.Dense(2,activation="relu"),
    layers.Dense(1,activation="sigmoid")
])
model.compile(
    optimizer="adam",
    loss = tfk.losses.BinaryCrossentropy(),
    metrics=["Accuracy"]
)

In [7]:
def drop_column_series_dataframe(input):
    col_list = ["BloodPressure","SkinThickness"]
    if isinstance(input,pd.DataFrame):
        input.drop(columns = col_list, axis=1,inplace=True)
    elif isinstance(input,pd.Series):
        input.drop(columns=col_list,inplace=True)

In [8]:
drop_redundant_features_transformer = FunctionTransformer(drop_column_series_dataframe)

In [9]:
preprocessor = make_pipeline(drop_redundant_features_transformer, scaler)

In [10]:
model_pipeline = Pipeline(steps=[
    ("preprocessor",preprocessor),
    ("model",model)
])

In [11]:
target = np.asarray(target).reshape(-1,1)

In [12]:
features

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age
0,6,148,72,35,0,33.6,0.627,50
1,1,85,66,29,0,26.6,0.351,31
2,8,183,64,0,0,23.3,0.672,32
3,1,89,66,23,94,28.1,0.167,21
4,0,137,40,35,168,43.1,2.288,33
...,...,...,...,...,...,...,...,...
763,10,101,76,48,180,32.9,0.171,63
764,2,122,70,27,0,36.8,0.340,27
765,5,121,72,23,112,26.2,0.245,30
766,1,126,60,0,0,30.1,0.349,47


In [13]:
model_pipeline.fit(features,target)

ValueError: Expected 2D array, got scalar array instead:
array=nan.
Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.